# Exercises for Apache Spark™ and Scala Workshops PART - II

This are my own solutions version in PySpark of the Exercises proposed by Jacek Laskowski in https://github.com/jaceklaskowski/spark-workshop/tree/gh-pages/exercises

## Exercises:

11. [Finding Most Populated Cities Per Country](#11) 
12. [Using upper Standard Function](#12)
13. [Difference in Days Between Dates As Strings](#13)
14. [Counting Occurences Of Years and Months For 24 Months From Now](#14) No es claro
15. [Why are all fields null when querying with schema?](#15) No es claro
16. [How to add days (as values of a column) to date?](#16)
17. [Calculating aggregations](#17) 
18. [Finding maximum values per group (groupBy)](#18)
19. [Collect values per group](#19)
20. [Multiple Aggregations](#20)

# SET UP

In [2]:
!pip install findspark

import findspark
findspark.init()

In [3]:
# Cargar Pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf


spark = SparkSession.builder.appName("Test_spark").master("local[*]").getOrCreate()

spark

# LIBRARIES

In [4]:
from pyspark.sql.functions import *
from pyspark.sql import Window
from pyspark.sql.types import *

## 11. Finding Most Populated Cities Per Country <a id='11'></a>

In [61]:
df_input = spark.createDataFrame([
    ["Warsaw","Poland","1 764 615"],
    ["Cracow","Poland","769 498"],
    ["Paris","France","2 206 488"],
    ["Villeneuve-Loubet","France","15 020"],
    ["Pittsburgh PA","United States","302 407"],
    ["Chicago IL","United States","2 716 000"],
    ["Milwaukee WI","United States","595 351"],
    ["Vilnius","Lithuania","580 020"],
    ["Stockholm","Sweden","972 647"],
    ["Goteborg","Sweden","580 020"]
],("name","country","population"))

df_input.show()

+-----------------+-------------+----------+
|             name|      country|population|
+-----------------+-------------+----------+
|           Warsaw|       Poland| 1 764 615|
|           Cracow|       Poland|   769 498|
|            Paris|       France| 2 206 488|
|Villeneuve-Loubet|       France|    15 020|
|    Pittsburgh PA|United States|   302 407|
|       Chicago IL|United States| 2 716 000|
|     Milwaukee WI|United States|   595 351|
|          Vilnius|    Lithuania|   580 020|
|        Stockholm|       Sweden|   972 647|
|         Goteborg|       Sweden|   580 020|
+-----------------+-------------+----------+



In [76]:
df_output = df_input.withColumn("population_int",regexp_replace(col("population"), " ", "").cast(IntegerType()))
df_output = df_output.orderBy(col("population_int").desc()).drop("population_int")
df_output.show()

+-----------------+-------------+----------+
|             name|      country|population|
+-----------------+-------------+----------+
|       Chicago IL|United States| 2 716 000|
|            Paris|       France| 2 206 488|
|           Warsaw|       Poland| 1 764 615|
|        Stockholm|       Sweden|   972 647|
|           Cracow|       Poland|   769 498|
|     Milwaukee WI|United States|   595 351|
|         Goteborg|       Sweden|   580 020|
|          Vilnius|    Lithuania|   580 020|
|    Pittsburgh PA|United States|   302 407|
|Villeneuve-Loubet|       France|    15 020|
+-----------------+-------------+----------+



## 12. Using upper Standard Function <a id='12'></a>

In [15]:
df_input = spark.createDataFrame([
[0,"Warsaw","Poland"],
[1,"Villeneuve-Loubet","France"],
[2,"Vranje","Serbia"],
[3,"Pittsburgh","US"]],("id","city","country"))

df_input.show()

+---+-----------------+-------+
| id|             city|country|
+---+-----------------+-------+
|  0|           Warsaw| Poland|
|  1|Villeneuve-Loubet| France|
|  2|           Vranje| Serbia|
|  3|       Pittsburgh|     US|
+---+-----------------+-------+



In [16]:
df_output = df_input.withColumn("upper_city",upper("city"))
df_output.show()

+---+-----------------+-------+-----------------+
| id|             city|country|       upper_city|
+---+-----------------+-------+-----------------+
|  0|           Warsaw| Poland|           WARSAW|
|  1|Villeneuve-Loubet| France|VILLENEUVE-LOUBET|
|  2|           Vranje| Serbia|           VRANJE|
|  3|       Pittsburgh|     US|       PITTSBURGH|
+---+-----------------+-------+-----------------+



## 13. Difference in Days Between Dates As Strings <a id='13'></a>

In [24]:
df_input = spark.createDataFrame([
   [1,"08/11/2015"],
   [2,"09/11/2015"],
   [3,"09/12/2015"]],("id","date_string"))

df_input.show()

+---+-----------+
| id|date_string|
+---+-----------+
|  1| 08/11/2015|
|  2| 09/11/2015|
|  3| 09/12/2015|
+---+-----------+



In [29]:
dateFormat = "dd/MM/yyyy"
df_output = df_input.withColumn("to_date",to_date(col("date_string"),dateFormat))
df_output = df_output.withColumn("diff",datediff(current_date(),col("to_date")))
df_output.show()

+---+-----------+----------+----+
| id|date_string|   to_date|diff|
+---+-----------+----------+----+
|  1| 08/11/2015|2015-11-08|1889|
|  2| 09/11/2015|2015-11-09|1888|
|  3| 09/12/2015|2015-12-09|1858|
+---+-----------+----------+----+



## 14. Counting Occurences Of Years and Months For 24 Months From Now <a id='14'></a>

In [8]:
df_input = spark.createDataFrame([
    [202001,500],
    [202001,600],
    [201912,100],
    [201910,200],
    [201910,100],
    [201909,400],
    [201601,5000]],("YEAR_MONTH","AMOUNT"))

df_input.show()

+----------+------+
|YEAR_MONTH|AMOUNT|
+----------+------+
|    202001|   500|
|    202001|   600|
|    201912|   100|
|    201910|   200|
|    201910|   100|
|    201909|   400|
|    201601|  5000|
+----------+------+



## 15. Why are all fields null when querying with schema? <a id='15'></a>

2019-07-22 00:10:15,030|10.29.2.6|
2019-07-22 00:10:15,334|10.1.198.41|
2019-07-22 00:10:15,400|10.1.198.41|
2019-07-22 00:10:15,511|10.1.198.41|
2019-07-22 00:10:16,911|10.1.198.41|

## 16. How to add days (as values of a column) to date? <a id='16'></a>

In [82]:
df_input = spark.createDataFrame([
  [0, "2016-01-1"],
  [1, "2016-02-2"],
  [2, "2016-03-22"],
  [3, "2016-04-25"],
  [4, "2016-05-21"],
  [5, "2016-06-1"],
  [6, "2016-03-21"]],("number_of_days", "date"))

df_input.show()

+--------------+----------+
|number_of_days|      date|
+--------------+----------+
|             0| 2016-01-1|
|             1| 2016-02-2|
|             2|2016-03-22|
|             3|2016-04-25|
|             4|2016-05-21|
|             5| 2016-06-1|
|             6|2016-03-21|
+--------------+----------+



In [84]:
# Set date format of the original string date
dateFormat = "yyyy-MM-d"
# Organize types
df_output = (df_input
             .withColumn("to_date",to_date(col("date"),dateFormat))
             .withColumn("number_of_days",col("number_of_days").cast(IntegerType())))
# Add days
df_output = df_output.withColumn("future", expr("date_add(to_date,number_of_days)"))
df_output.show()

+--------------+----------+----------+----------+
|number_of_days|      date|   to_date|    future|
+--------------+----------+----------+----------+
|             0| 2016-01-1|2016-01-01|2016-01-01|
|             1| 2016-02-2|2016-02-02|2016-02-03|
|             2|2016-03-22|2016-03-22|2016-03-24|
|             3|2016-04-25|2016-04-25|2016-04-28|
|             4|2016-05-21|2016-05-21|2016-05-25|
|             5| 2016-06-1|2016-06-01|2016-06-06|
|             6|2016-03-21|2016-03-21|2016-03-27|
+--------------+----------+----------+----------+



## 17. Calculating aggregations <a id='17'></a>

In [77]:
df_input = spark.createDataFrame([
    [0,'Warsaw','1 764 615'],
    [1,'Villeneuve-Loubet','15 020'],
    [2,'Vranje','83 524'],
    [3,'Pittsburgh','1 775 634']],("id","name","population"))

df_input.show()

+---+-----------------+----------+
| id|             name|population|
+---+-----------------+----------+
|  0|           Warsaw| 1 764 615|
|  1|Villeneuve-Loubet|    15 020|
|  2|           Vranje|    83 524|
|  3|       Pittsburgh| 1 775 634|
+---+-----------------+----------+



In [81]:
df_output = df_input.withColumn("population_int",regexp_replace(col("population"), " ", "").cast(IntegerType()))
df_output = df_output.agg(max("population_int").alias("population"))
df_output.show()

+----------+
|population|
+----------+
|   1775634|
+----------+



## 18. Finding maximum values per group (groupBy) <a id='18'></a>

In [53]:
df_input = spark.createDataFrame([
    [0,0],
    [1,1],
    [2,0],
    [3,1],
    [4,0]],("id","group"))

df_input.show()

+---+-----+
| id|group|
+---+-----+
|  0|    0|
|  1|    1|
|  2|    0|
|  3|    1|
|  4|    0|
+---+-----+



In [54]:
df_output = (df_input
             .groupby("group")
             .agg(max("id").alias("max_id")))
df_output.show()

+-----+------+
|group|max_id|
+-----+------+
|    0|     4|
|    1|     3|
+-----+------+



## 19. Collect values per group <a id='19'></a>

In [55]:
df_input = spark.createDataFrame([
    [0,0],
    [1,1],
    [2,0],
    [3,1],
    [4,0]],("id","group"))

df_input.show()

+---+-----+
| id|group|
+---+-----+
|  0|    0|
|  1|    1|
|  2|    0|
|  3|    1|
|  4|    0|
+---+-----+



In [56]:
df_output = df_input.groupby("group").agg(collect_set("id").alias("ids"))
df_output.show()

+-----+---------+
|group|      ids|
+-----+---------+
|    0|[0, 2, 4]|
|    1|   [1, 3]|
+-----+---------+



## 20. Multiple Aggregations <a id='20'></a>

In [57]:
df_input = spark.createDataFrame([
    [0,0],
    [1,1],
    [2,0],
    [3,1],
    [4,0]],("id","group"))

df_input.show()

+---+-----+
| id|group|
+---+-----+
|  0|    0|
|  1|    1|
|  2|    0|
|  3|    1|
|  4|    0|
+---+-----+



In [58]:
df_output = df_input.groupby("group").agg(max("id").alias("max_id"),min("id").alias("mind_id"))
df_output.show()

+-----+------+-------+
|group|max_id|mind_id|
+-----+------+-------+
|    0|     4|      0|
|    1|     3|      1|
+-----+------+-------+

